<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/wireless-pr3d/ep5g_latency_measurement_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [3]:
import re
import os
from getpass import getpass

In [4]:
with open('./sdr-test-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password: ············


Install required packages and dependencies. Ignore the warnings.

Import packages

In [5]:
import json
import time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

In [8]:
leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

[
    {
        "name": "adv-03-lease",
        "id": "30522f6f-e811-4ca8-8429-8651d8f8cf23",
        "reservation_id": "e94e4323-85ed-4605-a22f-8a6ccf660221",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:06:00.000000"
    },
    {
        "name": "worker-07-lease",
        "id": "38c4f00e-499f-4c97-8304-4662eebb57c9",
        "reservation_id": "4dc0eba9-4e45-4cca-9aa0-88c97132ccde",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:07:00.000000"
    },
    {
        "name": "worker-08-lease",
        "id": "938f25b0-2a4b-4f42-a0aa-49bff426fd8f",
        "reservation_id": "e1ea606e-b4af-47dc-8a69-8849fef9c8f5",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:08:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5",
        "reservation_id": "1a60b6a3-8276-4ba0-9167-658e643139b5",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:05:00.000000"
    }
]


In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [7]:
# ep5g reservation
ep5g_lease = reserve(
    { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": "100", "duration": { "days":7, "hours":0 } }
)

# advantech router reservation
adv3_lease = reserve(
    { "type":"network", "name": "adv-03", "net_name": "adv-03", "segment_id": "133", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker7_lease = reserve(
    { "type":"device", "name":"worker-07", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker8_lease = reserve(
    { "type":"device", "name":"worker-08", "duration": { "days":7, "hours":0 } }
)

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2023-11-03 15:05:25.023 | INFO     | chi.expeca:reserve:167 - reserving ep5g
2023-11-03 15:05:27.561 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5 to become "ACTIVE"
2023-11-03 15:05:32.603 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5 is PENDING.
2023-11-03 15:05:37.651 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5 is PENDING.
2023-11-03 15:05:42.693 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5 is PENDING.
2023-11-03 15:05:47.735 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5 is PENDING.
2023-11-03 15:05:52.775 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5 is P

[
    {
        "name": "adv-03-lease",
        "id": "30522f6f-e811-4ca8-8429-8651d8f8cf23",
        "reservation_id": "e94e4323-85ed-4605-a22f-8a6ccf660221",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:06:00.000000"
    },
    {
        "name": "worker-07-lease",
        "id": "38c4f00e-499f-4c97-8304-4662eebb57c9",
        "reservation_id": "4dc0eba9-4e45-4cca-9aa0-88c97132ccde",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:07:00.000000"
    },
    {
        "name": "worker-08-lease",
        "id": "938f25b0-2a4b-4f42-a0aa-49bff426fd8f",
        "reservation_id": "e1ea606e-b4af-47dc-8a69-8849fef9c8f5",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:08:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5",
        "reservation_id": "1a60b6a3-8276-4ba0-9167-658e643139b5",
        "status": "ACTIVE",
        "end_date": "2023-11-10T14:05:00.000000"
    }
]


In [10]:
leaseslist = list_reservations(brief=True)

adv3_lease = [lease for lease in leaseslist if lease["name"]=="adv-03-lease"][0]
worker7_lease = [lease for lease in leaseslist if lease["name"]=="worker-07-lease"][0]
worker8_lease = [lease for lease in leaseslist if lease["name"]=="worker-08-lease"][0]

ep5g_lease = [lease for lease in leaseslist if lease["name"]=="ep5g-lease"][0]
worker7_reservation_id = worker7_lease["reservation_id"]
worker8_reservation_id = worker8_lease["reservation_id"]

print(adv3_lease,ep5g_lease, worker7_reservation_id, worker8_reservation_id)

{'name': 'adv-03-lease', 'id': '30522f6f-e811-4ca8-8429-8651d8f8cf23', 'reservation_id': 'e94e4323-85ed-4605-a22f-8a6ccf660221', 'status': 'ACTIVE', 'end_date': '2023-11-10T14:06:00.000000'} {'name': 'ep5g-lease', 'id': 'b9ff0cf3-bf7a-4de2-9e28-4a1be19af3c5', 'reservation_id': '1a60b6a3-8276-4ba0-9167-658e643139b5', 'status': 'ACTIVE', 'end_date': '2023-11-10T14:05:00.000000'} 4dc0eba9-4e45-4cca-9aa0-88c97132ccde e1ea606e-b4af-47dc-8a69-8849fef9c8f5


In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).
It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [9]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

2023-11-03 15:11:53.260 | SUCCESS  | __main__:<module>:4 - edge-net is created.
2023-11-03 15:11:55.335 | SUCCESS  | __main__:<module>:8 - ep5g-vip-router router is created.
2023-11-03 15:11:55.336 | INFO     | __main__:<module>:9 - {
    "id": "40fd7322-eaa9-4d96-af51-039ba9b2daa0",
    "name": "ep5g-vip-router",
    "tenant_id": "f2831b1828814f5db878f45a61d1043c",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.115"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2023-11-03T14:11:53Z",
    "updated_at": "2023-11-03T14:11:55Z",
    "revision_number": 3,
    "proj

Now the network is ready to run the workloads. We start by running the edge-node perf-meas container.

In [11]:
edgenet = chi.network.get_network("edge-net")
chi.container.create_container(
    name = "edge-node",
    image = "gourav4871/perf-meas-full",
    reservation_id = worker8_reservation_id,
    environment = {"SERVER_DIR":"/mnt/volume/"},
    mounts = [
        {'source': 'edge-volume', 'destination': '/mnt/volume/'}
    ],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.70.70.3/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active("edge-node")
logger.success("created edge-node container.")

2023-11-03 15:14:09.581 | SUCCESS  | __main__:<module>:20 - created edge-node container.


Next, we run the end-node perf-meas container.

In [12]:
advnet = chi.network.get_network("adv-03-net")
chi.container.create_container(
    name = "end-node",
    image = "gourav4871/perf-meas-full",
    reservation_id = worker7_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active("end-node")
logger.success("created end-node container.")

2023-11-03 15:16:32.661 | SUCCESS  | __main__:<module>:17 - created end-node container.


Run uplink bandwidth test

In [13]:
command = "iperf3 -c 10.70.70.3 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-03 15:23:00.056 | INFO     | __main__:<module>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


Run downlink bandwidth test

In [ ]:
command = "iperf3 -c 172.16.0.96 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

In [20]:
status = get_container_status("edge-node")
if status:
    chi.container.destroy_container("edge-node")
    wait_until_container_removed("edge-node")


status = get_container_status("end-node")
if status:
    chi.container.destroy_container("end-node")
    wait_until_container_removed("end-node")

logger.info(f"stopped and removed all containers")

# find the router again
router = None
try:
    router = chi.network.get_router("ep5g-vip-router")
except Exception as ex:
    logger.info(f"could not find ep5g-vip-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success(f"removed all routers from router")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success(f"removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success(f"deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("edge-net")
except Exception as ex:
    logger.info(f"could not find edge-net.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success(f"deleted the edge-net")

# remove the leases
# unreserve_byid(ep5g_lease["id"])
# unreserve_byid(adv_lease["id"])
# unreserve_byid(worker_lease["id"])

2023-11-03 10:40:59.742 | INFO     | __main__:<module>:12 - stopped and removed all containers
2023-11-03 10:41:01.761 | SUCCESS  | __main__:<module>:24 - removed all routers from router
2023-11-03 10:41:01.838 | INFO     | __main__:<module>:28 - checking all 4 subnets.
2023-11-03 10:41:07.164 | SUCCESS  | __main__:<module>:34 - removed all subnets from router
2023-11-03 10:41:08.367 | SUCCESS  | __main__:<module>:37 - deleted the router
2023-11-03 10:41:13.496 | SUCCESS  | __main__:<module>:47 - deleted the edge-net


In [21]:
unreserve_byid(ep5g_lease["id"])
unreserve_byid(adv3_lease["id"])
unreserve_byid(worker7_lease["id"])
unreserve_byid(worker8_lease["id"])


2023-11-03 10:41:22.639 | INFO     | chi.expeca:remove_lease:84 - Removing ep5g-lease reservation with id 13454d25-bfcd-41a0-b387-d28cc1d55880.
2023-11-03 10:41:23.683 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id 13454d25-bfcd-41a0-b387-d28cc1d55880 to become "None"
2023-11-03 10:41:28.724 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 13454d25-bfcd-41a0-b387-d28cc1d55880 is None.
2023-11-03 10:41:28.725 | SUCCESS  | chi.expeca:try_to_remove:95 - done
2023-11-03 10:41:28.749 | INFO     | chi.expeca:remove_lease:84 - Removing adv-03-lease reservation with id bc3aac29-52cd-4ef7-85d3-5b0dc59c9ff9.
2023-11-03 10:41:41.714 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for adv-03-lease with id bc3aac29-52cd-4ef7-85d3-5b0dc59c9ff9 to become "None"
2023-11-03 10:41:46.756 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-03-lease with id bc3aac29-52cd-4ef7-85d3-5b0dc59c9ff9

In [26]:
ep5g_lease = [lease for lease in leaseslist if lease["name"]=="ep5g-lease"][0]
unreserve_byid(ep5g_lease["id"])
print(ep5g_lease)

2023-11-03 10:53:31.013 | INFO     | chi.expeca:remove_lease:84 - Removing ep5g-lease reservation with id a2b46dd0-4d0c-4387-b882-d4ec18fb461d.
2023-11-03 10:53:51.920 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id a2b46dd0-4d0c-4387-b882-d4ec18fb461d to become "None"
2023-11-03 10:53:56.958 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id a2b46dd0-4d0c-4387-b882-d4ec18fb461d is None.
2023-11-03 10:53:56.959 | SUCCESS  | chi.expeca:try_to_remove:95 - done


{'name': 'ep5g-lease', 'id': 'a2b46dd0-4d0c-4387-b882-d4ec18fb461d', 'reservation_id': 'e257b828-ef18-453c-812d-89de76929130', 'status': 'ACTIVE', 'end_date': '2023-11-09T20:19:00.000000'}
